**Mount Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from google.colab import drive
drive.mount('/content/drive')

!nvidia-smi

!ls /content/drive/MyDrive/data

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: nvidia-smi: command not found
192-168-5-106_2025-10-08-09-00-49-AM_ISPM_RolltoRoll_GEN5_ISPM_PH3_Reference_Both_Edge_Labels
192-168-5-106_2025-10-08-09-16-10-AM_ISPM_RolltoRoll_GEN5_ISPM_PH3_Increase_Both_Edge_Labels
192-168-5-106_2025-10-08-10-55-35-AM_ISPM_RolltoRoll_GEN5_ISPM_PH3_Decrease_Both_Edge_Labels
2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps
2025-05-27-03-22-58-PM_Gradually_Increasing_Pressure_0dot4mBar_Steps_ISPM_PH4
fit
preprocessed
ringdown


**1. Introduction**

In [ ]:
# Build 3D Acoustic Tensor from Replicate Data

# This notebook parses all `*_Sensing.json` files for a given replicate inside a folder,
# matches them with corresponding `*_NozzleCheck_*_mBar.bmp` images to extract pressure values,
# and builds a **3D tensor (m × n × k)** where:
#
# - m = number of nozzles (A–D combined)
# - n = number of samples per waveform (~180)
# - k = number of pressure levels (one per file)
#
# Each nozzle is labeled as A1, A2, …, D320, etc.


**2. Configuration**

In [ ]:
import os, re, json, glob
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from ipywidgets import interact, FloatRangeSlider

# ---------------- CONFIGURATION ----------------
folder = "/content/drive/MyDrive/data/2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps"  # Path to folder containing *_Sensing.json and *_NozzleCheck_*.bmp
replicate = 1       # Replicate number to process (1 or 2)
version = "lowering"        # either "lowering" or "increasing"
# ------------------------------------------------


**3. File matching**

In [ ]:
# Helper patterns (we must double the braces {{ }} so .format() doesn’t eat them)
json_pattern = re.compile(r"(?P<ts>\d{{4}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{6}})_{}_Sensing\.json$".format(replicate))
bmp_pattern = re.compile(r"(?P<ts>\d{{4}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{6}})_{}_NozzleCheck_[-+]?\d+(?:\.\d+)?_mBar\.bmp$".format(replicate))
pressure_pattern = re.compile(r"NozzleCheck_([-+]?\d+(?:\.\d+)?)_mBar", re.IGNORECASE)

# Gather files
json_files = sorted(glob.glob(os.path.join(folder, f"*_{replicate}_Sensing.json")))
bmp_files  = sorted(glob.glob(os.path.join(folder, f"*_{replicate}_NozzleCheck_*_mBar.bmp")))

print(f"Found {len(json_files)} sensing JSONs and {len(bmp_files)} BMP images for replicate {replicate}.")

# Build a timestamp -> pressure lookup from BMPs
pressure_lookup = {}
for bmp in bmp_files:
    name = os.path.basename(bmp)
    ts_match = re.match(r"(\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{6})", name)
    if not ts_match:
        continue
    ts = ts_match.group(1)
    m = pressure_pattern.search(name)
    if m:
        pressure = float(m.group(1))
        pressure_lookup[ts] = pressure

print(f"Pressure lookup built for {len(pressure_lookup)} timestamps.")



Found 96 sensing JSONs and 109 BMP images for replicate 1.
Pressure lookup built for 109 timestamps.


**4. Parse JSONs and build tensor**

In [ ]:
import errno
from google.colab import drive


def load_json_with_retry(path, retry_once=True):
    try:
        with open(path, "r") as f:
            return json.load(f)
    except OSError as e:
        # Drive mount disconnected
        if e.errno == 107 and retry_once:
            print("Drive disconnected. Remounting and retrying once...")
            drive.mount("/content/drive", force_remount=True)
            return load_json_with_retry(path, retry_once=False)
        raise

# Helper patterns (must double braces {{ }} so .format() doesn’t consume them)
json_pattern = re.compile(
    r"(?P<ts>\d{{4}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{6}})_{}_Sensing\.json$".format(replicate)
)
bmp_pattern = re.compile(
    r"(?P<ts>\d{{4}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{2}}-\d{{6}})_{}_NozzleCheck_[-+]?\d+(?:\.\d+)?_mBar\.bmp$".format(replicate)
)
pressure_pattern = re.compile(r"NozzleCheck_([-+]?\d+(?:\.\d+)?)_mBar", re.IGNORECASE)


# Gather files
json_files = sorted(glob.glob(os.path.join(folder, f"*_{replicate}_Sensing.json")))
bmp_files  = sorted(glob.glob(os.path.join(folder, f"*_{replicate}_NozzleCheck_*_mBar.bmp")))

print(f"Found {len(json_files)} sensing JSONs and {len(bmp_files)} BMP images for replicate {replicate}.")

# Build pressure lookup
pressure_lookup = {}
for bmp in bmp_files:
    name = os.path.basename(bmp)
    ts_match = re.match(r"(\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{6})", name)
    if not ts_match:
        continue
    ts = ts_match.group(1)
    m = pressure_pattern.search(name)
    if m:
        pressure = float(m.group(1))
        pressure_lookup[ts] = pressure

print(f"Pressure lookup built for {len(pressure_lookup)} timestamps.")

# --- Build Tensor ---
all_pressures = []
all_waveforms = []
bad_files = []  # store problematic JSONs

for js in json_files:
    ts = re.search(r"(\d{4}-\d{2}-\d{2}-\d{2}-\d{2}-\d{2}-\d{6})", os.path.basename(js)).group(1)
    pressure = pressure_lookup.get(ts)
    if pressure is None:
        print(f" No matching BMP pressure for {js}, skipping.")
        continue

    try:
        data = load_json_with_retry(js)
    except json.JSONDecodeError as e:
        print(f" JSON error in {js}: {e}")
        bad_files.append(js)
        continue
    except OSError as e:
        print(f" OS error in {js}: {e}")
        bad_files.append(js)
        continue


    combined_waveforms = []
    for bank_label in ['A', 'B', 'C', 'D']:
        key = f'Bank_{bank_label}'
        if key not in data:
            continue
        for nozzle_idx, waveform in enumerate(data[key], start=1):
            combined_waveforms.append(waveform)

    all_pressures.append(pressure)
    all_waveforms.append(np.array(combined_waveforms))

# Sort and stack
sorted_idx = np.argsort(all_pressures)
all_pressures = np.array(all_pressures)[sorted_idx]
all_waveforms = [all_waveforms[i] for i in sorted_idx]

tensor = np.stack(all_waveforms, axis=2)
nozzles = [f"{b}{i}" for b in ['A','B','C','D'] for i in range(1, len(all_waveforms[0])//4 + 1)]

print(f"\n Tensor built: shape = {tensor.shape} (nozzles × samples × pressures)")
print(f"Pressures: {all_pressures}")
print(f"Waveform lengths: {[len(row) for wf_set in all_waveforms for row in wf_set][0]} samples each")

if bad_files:
    print(f"\n Skipped {len(bad_files)} corrupted JSON files:")
    for bf in bad_files:
        print(f"   - {os.path.basename(bf)}")


lengths = [len(row) for wf_set in all_waveforms for row in wf_set]
print(f"Waveform lengths range: {min(lengths)} – {max(lengths)}")



Found 96 sensing JSONs and 109 BMP images for replicate 1.
Pressure lookup built for 109 timestamps.
 JSON error in /content/drive/MyDrive/data/2025-05-26-03-00-51-PM_Gradually_Lowering_Pressure_0dot4mBar_Steps/2025-05-27-14-50-58-624535_1_Sensing.json: Expecting ',' delimiter: line 114470 column 19 (char 2270956)

 Tensor built: shape = (1280, 124, 95) (nozzles × samples × pressures)
Pressures: [-87.6 -87.2 -86.8 -86.4 -86.  -85.6 -85.2 -84.8 -84.4 -84.  -83.6 -83.2
 -82.8 -82.4 -82.  -81.6 -81.2 -80.8 -80.4 -80.  -79.6 -79.2 -78.8 -78.4
 -78.  -77.6 -77.2 -76.8 -76.4 -76.  -75.6 -75.2 -74.8 -74.4 -74.  -73.6
 -73.2 -72.8 -72.4 -72.  -71.6 -71.2 -70.8 -70.4 -70.  -69.6 -69.2 -68.8
 -68.4 -68.  -67.6 -67.2 -66.8 -66.4 -66.  -65.6 -65.2 -64.8 -64.4 -64.
 -63.6 -63.2 -62.8 -62.4 -62.  -61.6 -61.2 -60.8 -60.4 -60.  -59.6 -59.2
 -58.8 -58.4 -58.  -57.6 -57.2 -56.8 -56.4 -56.  -55.6 -55.2 -54.8 -54.4
 -54.  -53.6 -53.2 -52.8 -52.4 -52.  -51.6 -51.2 -50.8 -50.4 -50. ]
Waveform lengths: 124 s

**5. Save results**

In [ ]:
out_tensor = f"/content/drive/MyDrive/data/preprocessed/tensor_replicate_{replicate}_{version}.npy"
out_press  = f"/content/drive/MyDrive/data/preprocessed/pressures_replicate_{replicate}_{version}.json"
out_nozz   = f"/content/drive/MyDrive/data/preprocessed/nozzles_replicate_{replicate}_{version}.json"

np.save(out_tensor, tensor.tolist())
with open(out_press, 'w') as f: json.dump(all_pressures.tolist(), f, indent=2)
with open(out_nozz, 'w') as f: json.dump(nozzles, f, indent=2)

print(f"Saved: {out_tensor}, {out_press}, {out_nozz}")


Saved: /content/drive/MyDrive/data/preprocessed/tensor_replicate_1_lowering.npy, /content/drive/MyDrive/data/preprocessed/pressures_replicate_1_lowering.json, /content/drive/MyDrive/data/preprocessed/nozzles_replicate_1_lowering.json


**6. Visualization example**

In [ ]:
# --- Colab fixes (run once per runtime) ---
# !pip -q install ipywidgets plotly

from google.colab import output
output.enable_custom_widget_manager()

import plotly.io as pio
pio.renderers.default = "colab"

# --- Visualization: waveform evolution across pressures for one nozzle ---
import os, json, glob, re
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from ipywidgets import FloatRangeSlider, interact

# Configuration
replicate = 1
target_nozzle = "B36"
version = "lowering"

# Paths for preprocessed data
base_dir = "/content/drive/MyDrive/data/preprocessed"
tensor_path = os.path.join(base_dir, f"tensor_replicate_{replicate}_{version}.npy")
press_path  = os.path.join(base_dir, f"pressures_replicate_{replicate}_{version}.json")
nozz_path   = os.path.join(base_dir, f"nozzles_replicate_{replicate}_{version}.json")

if not all(os.path.exists(p) for p in [tensor_path, press_path, nozz_path]):
    print(f"Missing preprocessed files for replicate {replicate}. Please generate them first.")
else:
    tensor = np.load(tensor_path)  # shape: (nozzles, samples, pressures)
    with open(press_path, "r") as f:
        all_pressures = np.array(json.load(f), dtype=float)
    with open(nozz_path, "r") as f:
        nozzles = json.load(f)

    print(f"Loaded replicate {replicate}: tensor {tensor.shape}, {len(nozzles)} nozzles, {len(all_pressures)} pressures.")

    if target_nozzle not in nozzles:
        print(f"Nozzle {target_nozzle} not found.")
    else:
        idx = nozzles.index(target_nozzle)

        slider = FloatRangeSlider(
            value=[float(all_pressures.min()), float(all_pressures.max())],
            min=float(all_pressures.min()),
            max=float(all_pressures.max()),
            step=0.4,
            description="Pressure range:",
            continuous_update=False,
            layout={"width": "85%"}
        )

        def plot_pressure_range(pressure_range, max_traces=20):
            pmin, pmax = pressure_range
            mask = (all_pressures >= pmin) & (all_pressures <= pmax)
            selected_indices = np.where(mask)[0]
            selected_pressures = all_pressures[mask]

            if len(selected_indices) == 0:
                print("No pressures in range.")
                return

            # cap traces so Plotly renders reliably in Colab
            if len(selected_indices) > max_traces:
                keep = np.linspace(0, len(selected_indices)-1, max_traces, dtype=int)
                selected_indices = selected_indices[keep]
                selected_pressures = selected_pressures[keep]

            fig = go.Figure()
            x = np.arange(tensor.shape[1])

            for i, p in zip(selected_indices, selected_pressures):
                fig.add_trace(go.Scatter(
                    x=x,
                    y=tensor[idx, :, i],
                    mode="lines",
                    name=f"{p:.1f} mBar",
                    hovertemplate="Pressure: %{text} mBar<br>Sample: %{x}<br>Amplitude: %{y}<extra></extra>",
                    text=[f"{p:.1f}"] * len(x),
                ))

            fig.update_layout(
                title=f"Waveform evolution — Nozzle {target_nozzle} ({replicate}_{version})",
                xaxis_title="Sample index",
                yaxis_title="Amplitude",
                height=520,
                margin=dict(l=40, r=220, t=60, b=40),
                legend=dict(font=dict(size=9))
            )

            fig.show(renderer="colab")

        interact(plot_pressure_range, pressure_range=slider);


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.9 MB/s eta 0:00:00
Loaded replicate 1: tensor (1280, 124, 95), 1280 nozzles, 95 pressures.


interactive(children=(FloatRangeSlider(value=(-87.6, -50.0), continuous_update=False, description='Pressure ra…